#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [9]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 800 kB 5.1 MB/s 
     |████████████████████████████████| 512 kB 48.9 MB/s 
     |████████████████████████████████| 125 kB 78.6 MB/s 


Скачаем данные

In [10]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

total 12
drwx------ 4 root root 4096 Nov 26 08:32 data
drwxr-xr-x 3 root root 4096 Dec 18 14:12 __MACOSX
drwxr-xr-x 1 root root 4096 Dec 16 00:01 sample_data


Посмотрим как выглядит один из файлов разметки

In [11]:
!cat data/train/apple_1.xml

<annotation>
	<folder>train</folder>
	<filename>apple_1.jpg</filename>
	<path>C:\tensorflow1\models\research\object_detection\images\train\apple_1.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>0</width>
		<height>0</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>apple</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>8</xmin>
			<ymin>15</ymin>
			<xmax>331</xmax>
			<ymax>349</ymax>
		</bndbox>
	</object>
</annotation>


## Релизуйте выборку для YoloV1 - 2 балла

In [12]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict
import xml.etree.ElementTree as ET

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc

# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [13]:
"data/" + '/'.join(r'C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg'.split('\\')[-2:])

'data/train/apple_3.jpg'

In [14]:
p = "data/" + '/'.join(r'C:\tensorflow1\models\research\object_detection\images\train\apple_3.jpg'.split('\\')[-2:])
Image.open(p).size

(1000, 708)

In [15]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, S, B, C, transforms=None):
        self.image_paths = sorted(glob.glob(os.path.join(data_dir, "*.jpg")))  ## YOUR CODE
        self.box_paths = sorted(glob.glob(os.path.join(data_dir, "*.xml")))  ## YOUR CODE

        assert len(self.image_paths) == len(self.box_paths)

        self.transforms = transforms
        self.S = S
        self.B = B
        self.C = C

    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx])

        if self.transforms:
          transformed = self.transforms(image=image, bboxes=boxes, class_labels=class_labels)
          image = transformed['image']
          boxes = transformed['bboxes']
          class_labels = transformed['class_labels']
        
        ## YOUR CODE
        target_tensor = torch.zeros(self.S, self.S, 5 * self.B + self.C, dtype=float)

        for ind in range(len(boxes)):
          x_cen = boxes[ind][0] * image.shape[0] # посчитали реальный центр координаты x 
          y_cen = boxes[ind][1] * image.shape[1] # посчитали реальный центр координаты y
          # определим ячейку, к которой относится бокс
          sw = int(x_cen // (image.shape[0] // self.S)) 
          sh = int(y_cen // (image.shape[1] // self.S))
          for i in range(4, 5 * self.B, 5):
            # если в таргете (первая пятерка) еще не заполнен конфиденс, то заполняем его нашим боксом, иначе переходим к следующей пятерке
              if target_tensor[sw, sh, i] == 0:
                target_tensor[sw, sh, i - 4] = boxes[ind][0]
                target_tensor[sw, sh, i - 3] = boxes[ind][1]
                target_tensor[sw, sh, i - 2] = boxes[ind][2]
                target_tensor[sw, sh, i - 1] = boxes[ind][3]
                target_tensor[sw, sh, i] = 1  # уверенность для этого бокса = 1
                target_tensor[sw, sh, 5 * self.B + class_labels[ind] - 1] = 1 # вероятность класса в этом боксе = 1
                break

        return torch.from_numpy(image).type(torch.float32), target_tensor

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      ## YOUR CODE
      boxdict = xmltodict.parse(ET.tostring(ET.parse(xml_filename).getroot()))

      filepath = "data/" + '/'.join(boxdict['annotation']['path'].split('\\')[-2:])
      width, height = Image.open(filepath).size

      if type(boxdict['annotation']['object']) is dict:
        obj = boxdict['annotation']['object']
        box = [int(obj['bndbox']['xmin']),
                        int(obj['bndbox']['ymin']),
                        int(obj['bndbox']['xmax']),
                        int(obj['bndbox']['ymax']),
                        ]
        yolo_box = self.__convert_to_yolo_box_params(box, width, height)
        boxes.append(yolo_box)
        class_labels.append(class2tag[obj['name']])
      else:
        for obj in boxdict['annotation']['object']:
          box = [int(obj['bndbox']['xmin']),
                        int(obj['bndbox']['ymin']),
                        int(obj['bndbox']['xmax']),
                        int(obj['bndbox']['ymax']),
                        ]
          yolo_box = self.__convert_to_yolo_box_params(box, width, height)
          boxes.append(yolo_box)
          class_labels.append(class2tag[obj['name']])
      ## YOUR CODE

      return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = []

      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
    
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
      return ans

In [16]:

WIDTH, HEIGHT = 448, 448

train_transform = A.Compose([A.Resize(width=WIDTH, height=HEIGHT),
                             A.HorizontalFlip(p=0.5),
                             A.RandomBrightnessContrast(p=0.2)], ## YOUR CODE
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
test_transform = A.Compose([A.Resize(width=WIDTH, height=HEIGHT),
                            A.HorizontalFlip(p=0.5),
                            A.RandomBrightnessContrast(p=0.2)], ## YOUR CODE
                            bbox_params=A.BboxParams(format='yolo',
                                                     label_fields=['class_labels']))

In [17]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train", 
     S = 7, B = 2, C = 3
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test",
    S = 7, B = 2, C = 3
    )

In [18]:
# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [19]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4, 
    shuffle=False
    )

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [20]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [21]:
a = torch.permute(train_dataset[0][0], (2, 1, 0))
torch.unsqueeze(a, dim=0).size()

torch.Size([1, 3, 448, 448])

In [22]:
train_dataset[0][0].dim()

3

In [23]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2, stride = 2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C

        self.conv1 =  CNNBlock(in_channels = self.C, out_channels=64, kernel_size=7,  is_max_pool=True, stride = 2)
        self.conv2 =  CNNBlock(in_channels = 64, out_channels=128, kernel_size=5, is_max_pool=True, stride = 2)
        # self.conv3 =  CNNBlock(in_channels = 256, out_channels=512, kernel_size=1)
        # self.conv4 =  CNNBlock(in_channels = 128, out_channels=256, kernel_size=3)
        # self.conv5 =  CNNBlock(in_channels = 256, out_channels=1024, kernel_size=1)
        # self.conv4 =  CNNBlock(in_channels = 128, out_channels=256, kernel_size=3)
        # self.conv5 =  CNNBlock(in_channels = 256, out_channels=256, kernel_size=1)
        # self.conv6 =  CNNBlock(in_channels = 256, out_channels=1024, kernel_size=3, is_max_pool=True, stride = 2)

        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(in_features=86528, out_features=1024)
        self.linear2 = torch.nn.Linear(in_features=1024, out_features=self.S*self.S*(5*self.B + self.C))
        ## YOUR CODE

    def forward(self, x):
        #The eval() function takes a string expression as input, parses it, and returns the output after evaluating the input expression.
        ## YOUR CODE
        if x.dim() == 3:
          x = torch.unsqueeze(x, dim=-1)
          x = x.permute(3, 2, 1 ,0)
        else:
          x = x.permute(0, 3, 2, 1)

        for i in range(1, 3):
          x = eval(f'self.conv{i}(x)')

        x = self.flatten(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x.reshape(-1, self.S, self.S, self.C + self.B * 5)


# Убедитесь на одном изображении, что предсказания вашей модели имеют верное количество значений
temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

testing_image = train_dataset[0][0]
# testing_image = torch.permute(train_dataset[0][0], (2, 1, 0))
assert temp_model(testing_image).reshape(-1).shape[0] == expected_output_shape

In [35]:
for batch in train_dataloader:
  print(batch[0].shape, batch[1].shape)
  print(temp_model(batch[0]).shape)
  break

torch.Size([4, 448, 448, 3]) torch.Size([4, 7, 7, 13])
torch.Size([4, 7, 7, 13])


In [36]:
for pred, targ in train_dataloader:
  print(pred.shape)
  print(targ.shape)
  break

torch.Size([4, 448, 448, 3])
torch.Size([4, 7, 7, 13])


## Реализуйте YoloLoss - 3 балла

In [26]:
# перепишем iou, чтобы он принимал на вход тензор предстказаний и таргетов с координатами для каждой ячейки в каждом батче [x, y, w, h]
def intersection_over_union_tensor(predictions, target) -> torch.tensor:
  # prediction - это батч, который приходит в формате ([4, 7, 7, 4])
  pred_xmin = predictions[:, :, :, 0:1] - target[:, :, :, 2:3] / 2
  pred_ymin = predictions[:, :, :, 1:2] - target[:, :, :, 3:4] / 2
  pred_xmax = predictions[:, :, :, 0:1] + target[:, :, :, 2:3] / 2
  pred_ymax = predictions[:, :, :, 1:2] + target[:, :, :, 3:4] / 2
  targ_xmin = predictions[:, :, :, 0:1] - target[:, :, :, 2:3] / 2
  targ_ymin = predictions[:, :, :, 1:2] - target[:, :, :, 3:4] / 2
  targ_xmax = predictions[:, :, :, 0:1] + target[:, :, :, 2:3] / 2
  targ_ymax = predictions[:, :, :, 1:2] + target[:, :, :, 3:4] / 2

  x1 = torch.max(pred_xmin, targ_xmin)
  y1 = torch.max(pred_ymin, targ_ymin)
  x2 = torch.min(pred_xmax, targ_xmax)
  y2 = torch.min(pred_ymax, targ_ymax)

  # если разность координат отрицательная, значит боксы не пересекаются, ставим 0
  intersection_area = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)

  area_pred = (pred_xmax - pred_xmin) * (pred_ymax - pred_ymin)
  area_targ = (targ_xmax - targ_xmin) * (targ_ymax - targ_ymin)

  union_area = area_pred + area_targ - intersection_area

  iou = intersection_area / union_area
  return iou

In [27]:
t1 =  torch.randint(high=10, size=(4, 7, 7, 4))
t2 = torch.randint(high=10, size=(4, 7, 7, 4))
targ =  torch.randint(high=10, size=(4, 7, 7, 4))
b1 = intersection_over_union_tensor(t1, targ)
b2 = intersection_over_union_tensor(t2, targ)
ious = torch.cat([b1.unsqueeze(0), b2.unsqueeze(0)], dim=0) 
ious.shape

torch.Size([2, 4, 7, 7, 1])

In [28]:
targ =  torch.randint(high=10, size=(4, 7, 7, 13))
targ[:, :, :, 4:5].shape

torch.Size([4, 7, 7, 1])

In [29]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
      # prediction - это батч, который приходит в формате ([4, 7, 7, 13]), где 4 это размер батча

      # для первой части лосса нам необходимо научиться определять, является ли бокс j ответсвенным за ячейку i 
      # берем каждый объект в батче, в объекте рассматриваем каждую ячейку и в ней считаем iou обоих боксов с таргетом
      iou_b1_targ = intersection_over_union_tensor(predictions[:, :, :, 0:4], target[:, :, :, 0:4])
      iou_b2_targ = intersection_over_union_tensor(predictions[:, :, :, 5:9], target[:, :, :, 0:4])
      # получили тензоры размера ([4, 7, 7, 1])

      # теперь хотим для каждой ячейки выбрать объект с максимальным iou
      max_ious = torch.maximum(iou_b1_targ, iou_b2_targ)
      # и получить индекс этого объекта. Если 0 то максимальный iou у нулевого бокса (b1)
      best_boxes = (iou_b2_targ == max_ious).type(torch.float32)

      # и нужно знать ответсвенна ли ячейка за объект или нет (то есть вытащить конфиденс в таргете)
      resp_block = target[:, :, :, 4:5]

      # вытаскиваем координаты ответсвенных боксов с учетом ответсвенности ячейки (по сути это то как модель предсказала координаты)
      good_boxes = resp_block * (
          best_boxes * predictions[:, :, :, 5:9] 
          + (1 - best_boxes) * predictions[:, :, :, 0:4]
      ) # размер good_boxes ([4, 7, 7, 4])

      good_targets = resp_block * target[:, :, :, 0:4]

      # вычислим корень из ширины и высоты
      good_boxes[:, :, :, 2:4] = torch.sign(good_boxes[:, :, :, 2:4]) * torch.sqrt(
            torch.abs(good_boxes[:, :, :, 2:4] + 1e-6)
        )
      good_targets[:, :, :, 2:4] = torch.sign(good_targets[:, :, :, 2:4]) * torch.sqrt(
            torch.abs(good_targets[:, :, :, 2:4] + 1e-6)
        )

      # посчитаем первую часть лосса, то есть mse для координат центра, высоты и ширины (1 и 2 слагаемые)
      loss_1_2 = self.lambda_coord * self.mse(
            torch.flatten(good_boxes, end_dim=-2),
            torch.flatten(good_targets, end_dim=-2),
        )

      # посчитаем разность mse для конфиденсов (третье слагаемое) для ответсвенных боксов
      conf_pred_box = resp_block * (
            best_boxes * predictions[:, :, :, 9:10] + (1 - best_boxes) * predictions[:, :, :, 4:5]
        )

      loss3 = self.lambda_coord * self.mse(
            torch.flatten(conf_pred_box),
            torch.flatten(resp_block),
        )
      # посчитаем разность mse для конфиденсов (четвертое слагаемое)
      loss4 = (self.mse(
            torch.flatten((1 - resp_block) * predictions[:, :, :, 0:4]),
            torch.flatten((1 - resp_block) * target[:, :, :, 0:4]),
        ) + self.mse(
            torch.flatten((1 - resp_block) * predictions[:, :, :, 5:9]),
            torch.flatten((1 - resp_block) * target[:, :, :, 0:4]),
        )) * self.lambda_noobj

      # последняя часть лосса, mse для вероятностей класса
      loss5 = self.mse(
            torch.flatten(resp_block * predictions[:, :, :, 10::]),
            torch.flatten(resp_block * target[:, :, :, 10::]),
        )
      return loss_1_2 + loss3 + loss4 + loss5

## Реализуйте дополнительные функции из статьи - 2 балла

In [30]:
def non_max_suppression(bboxes, iou_threshold, threshold):
  '''
  bboxes list[lists] - [x_min, y_min, x_max, y_max, conf, class]
  '''
  # оставляем боксы с примелемым конфиденсом
  good_bboxes = []
  for box in bboxes:
    if box[-2] > threshold:
      good_bboxes.append(box)
  
  # Сортируем боксы по убыванию конфиденса
  good_bboxes = sorted(good_bboxes, key=lambda x: x[1], reverse=True)
  res_bboxes = list()
  while len(good_bboxes) > 0:
    # достаем элемент с максимальной уверенностью и удаляем его из списка
    fix_box = good_bboxes.pop(0)
    saved_box = []
    for box in good_bboxes:
      # если классы не совпадают, то сохраняем бокс
      if fix_box[-1] != box[-1]:
        saved_box.append(box)
      # если классы совпадают, то оставляем объект только в случае низкого iou
      elif intersection_over_union(torch.tensor(fix_box[:-2]), torch.tensor(box[:-2])) < iou_threshold:
        saved_box.append(box)

      # в результат просто записываем бокс с максимальной уверенностью
      res_bboxes.append(fix_box)

  return res_bboxes

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    
    model.eval()
    # в даталоадере лежат сырые данные и таргеты по батчам, итерируемся по ним
    # размер pred ([4, 448, 448, 3]), размер target ([4, 7, 7, 13])
    for raw, target in loader:

      with torch.no_grad():
            predictions = model(raw)
      # теперь predictions имеет такую же размерность как и таргет ([4, 7, 7, 13])

      # теперь хотим применить non_max_suppression для всех боксов и оставить только те, которые имеют допустимый конфиденс

      # в общем я не знаю как реализовывать эти танцы с бубнами
    
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [38]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        images, target = train_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx) -> None:
        images, target = val_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        self.log("val_loss", loss, prog_bar=True)

In [39]:
model = YOLO
n_epochs = 5

yolo_learner = YOLOLearner()  

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | YOLO     | 89.5 M
1 | loss  | YoloLoss | 0     
-----------------------------------
89.5 M    Trainable params
0         Non-trainable params
89.5 M    Total params
357.894   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Итак, расскажу что тут происходило. Во первых стоит отметить, что начиная с написания лосса я опиралась на [вот этот гитхаб](https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/Pytorch/object_detection/YOLO). Копипаста, конечно, нет, но достаточно много идей было взято оттуда.


Долго разбиралась с тем как работать только с операциями torch, чтобы реализовался back-propogation, наверху в ячейках перед лосом выводила примеры операций и размерности, которые получаются после них.

Функция обучения стандартная, просто чтобы проверить, что лосс не падает :) Модель обучается, но плохо, а если добавить больше слоев, то коллаб падает по ОЗУ

В общем, я очень старалась, можно пожалуйста что-нибудь около 6...

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [56]:
image, targets = next(iter(val_dataset))
preds = yolo_learner(image)

In [54]:
preds.shape

torch.Size([1, 7, 7, 13])

In [57]:
image.shape

torch.Size([448, 448, 3])

In [50]:
from PIL import ImageDraw
image = image.permute(2, 1, 0)
image = torchvision.transforms.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image

ValueError: ignored